# Question Classification with DistilBert

Different Classes
1.   HUM for questions about humans
2.   ENTY for questions about entities
3.   DESC for questions asking you for a description
4.   NUM for questions where the answer is numerical
5.   LOC for questions where the answer is a location
6.   ABBR for questions asking about abbreviations

# 1. Import Libraries

In [4]:
# Install Transformers Package
!pip install transformers

     |████████████████████████████████| 2.5MB 7.5MB/s 
     |████████████████████████████████| 901kB 46.3MB/s 
     |████████████████████████████████| 3.3MB 49.1MB/s 


In [5]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

import numpy as np
from torchtext.legacy import data
from torchtext.legacy import datasets
import torch.nn.functional as F

# Preliminaries

from torchtext.legacy.data import Field, LabelField, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertModel

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# 2. Preprocessing--Build Fileds, Create Vocab and Dataloaders

In [6]:
# Load Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [7]:
# Decalre max sequence length
MAX_SEQ_LEN = 64

# Create fields
text_field = Field(sequential=True, use_vocab=False, tokenize=tokenizer.encode, batch_first=True,
                   fix_length = MAX_SEQ_LEN, pad_token=tokenizer.pad_token_id, unk_token=tokenizer.unk_token_id)
label_field = LabelField(sequential=False, batch_first=True)

# Split dataset with fields
train_data, test_data = datasets.TREC.splits(text_field, label_field)
train_data, valid_data = train_data.split([0.8, 0.2])

downloading train_5500.label


train_5500.label: 100%|██████████| 336k/336k [00:00<00:00, 1.32MB/s]


downloading TREC_10.label


TREC_10.label: 100%|██████████| 23.4k/23.4k [00:00<00:00, 362kB/s]


In [8]:
# Create vocab for label_field
label_field.build_vocab(train_data)
#vars(label_field.vocab)

In [9]:
# Create dataloaders with datasets
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                            batch_size = BATCH_SIZE, 
                                                                            device = device)

# 3. Model Implementation

In [10]:
class Transformer(nn.Module):
    def __init__(self, output_dim, freeze=True):
        super().__init__()
        self.transformer = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.hidden_dim = int(768)
        self.fc = nn.Linear(self.hidden_dim, 256)
        self.linear = nn.Linear(256, output_dim)
        
        if freeze:
            for param in self.transformer.parameters():
                param.requires_grad = False
        
    def forward(self, ids):
        # ids = [batch size, seq len]
        output = self.transformer(ids, output_attentions=True)
        hidden = output.last_hidden_state
        # hidden = [batch size, seq len, hidden dim]
        attention = output.attentions[-1]
        # attention = [batch size, n heads, seq len, seq len]
        cls_hidden = hidden[:,0,:]
        fced = F.relu(self.fc(torch.tanh(cls_hidden)))
        # prediction = [batch size, output dim]
        return self.linear(fced)

In [12]:
# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 4. Define Training and Evaluating Functions

In [13]:
# Define Train and Evaluate Function

def train(model, device, train_loader, valid_loader, optimizer, criterion, epoch, log_interval=1, dry_run=False, save=False):
    model.to(device)
    criterion.to(device)
    model.train()
    best_loss = np.inf
    for ep in range(1, epoch+1):
        current = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            current += len(data)
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, current, len(train_loader.dataset),
                    100. * current / len(train_loader.dataset), loss.item()))
                if dry_run:
                    break
        loss, _, _, _ = test(model, device, valid_loader, nn.CrossEntropyLoss(reduction='sum'), validation=True)
        if save:
            if loss < best_loss:
                best_loss = loss
                torch.save(model.state_dict(), 'checkpoint.pt')
                print('***Model Updated***')

def test(model, device, test_loader, criterion, validation=False):
    model.to(device)
    criterion.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    wrongx = []
    wrongy = []
    truey = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # test_loss += criterion(output, target, reduction='sum').item()  # sum up batch loss
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            selector = ~pred.eq(target.view_as(pred)).squeeze(1)
            # print(selector.shape)
            wrongx.append(data[selector])
            wrongy.append(pred[selector])
            truey.append(target[selector])
            #print(selector.shape)
            #print(data[selector].shape)

    test_loss /= len(test_loader.dataset)
    if validation:
        print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
    else:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
    
    wrongx = torch.cat( wrongx, dim=0 )
    wrongy = torch.cat( wrongy, dim=0 )
    truey = torch.cat( truey, dim=0 )
    return test_loss, wrongx, wrongy, truey

# 5. Train, Test, and Analysis

In [14]:
# Create Model
model = Transformer(output_dim=6, freeze=False)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
print(f'The model has {count_parameters(model):,} trainable parameters')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 66,561,286 trainable parameters


In [20]:
# -----------Run only to load model-----------
# LOAD_PATH = 'checkpoint.pt'
# model.load_state_dict(torch.load(LOAD_PATH))
# --------------------------------------------

<All keys matched successfully>

In [15]:
# Train
EPOCH=5
train(model, device, train_iterator, valid_iterator, optimizer, criterion, EPOCH, save=True)
a = test(model, device, test_iterator, nn.CrossEntropyLoss(reduction='sum'))

Train Epoch: 1 [128/4362 (3%)]	Loss: 1.814526
Train Epoch: 1 [256/4362 (6%)]	Loss: 1.782116
Train Epoch: 1 [384/4362 (9%)]	Loss: 1.762753
Train Epoch: 1 [512/4362 (12%)]	Loss: 1.740350
Train Epoch: 1 [640/4362 (15%)]	Loss: 1.737616
Train Epoch: 1 [768/4362 (18%)]	Loss: 1.710596
Train Epoch: 1 [896/4362 (21%)]	Loss: 1.706845
Train Epoch: 1 [1024/4362 (23%)]	Loss: 1.695476
Train Epoch: 1 [1152/4362 (26%)]	Loss: 1.697481
Train Epoch: 1 [1280/4362 (29%)]	Loss: 1.651254
Train Epoch: 1 [1408/4362 (32%)]	Loss: 1.675302
Train Epoch: 1 [1536/4362 (35%)]	Loss: 1.693456
Train Epoch: 1 [1664/4362 (38%)]	Loss: 1.640403
Train Epoch: 1 [1792/4362 (41%)]	Loss: 1.625218
Train Epoch: 1 [1920/4362 (44%)]	Loss: 1.649726
Train Epoch: 1 [2048/4362 (47%)]	Loss: 1.644765
Train Epoch: 1 [2176/4362 (50%)]	Loss: 1.642794
Train Epoch: 1 [2304/4362 (53%)]	Loss: 1.575946
Train Epoch: 1 [2432/4362 (56%)]	Loss: 1.607520
Train Epoch: 1 [2560/4362 (59%)]	Loss: 1.618702
Train Epoch: 1 [2688/4362 (62%)]	Loss: 1.624473
Tr

In [16]:
def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = tokenizer(sentence)['input_ids']
    if len(tokenized) < min_len:
        tokenized += [tokenizer.pad_token_id] * (128 - len(tokenized))
    tensor = torch.LongTensor(tokenized).to(device)
    tensor = tensor.unsqueeze(0)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [17]:
# Test custom questions here
sentence = "What does WYSIWYG stand for?"
label_field.vocab.itos[predict_class(model, sentence)]

'ABBR'

In [ ]:
'''
Configurations:
    Epoch = 5
    lr = 2e-5
    Accuracy on testset: 95.80 %
'''